In [1]:
import pandas as pd

df_clean = pd.read_pickle("df_clean_v1_07122021.pkl")
print(df_clean.shape)
df_clean.head()

(16068, 18)


,lyrics,explicit,song_name,song_popularity,mode,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,release_year,genres,genre,artist
song_id,,,,,,,,,,,,,,,,,,
3e9HZxeyfWwjeyPAMmWSSQ,Thought I'd end up with Sean. But he wasn't a ...,1,"thank u, next",86,1,0.22900,0.717,0.653,0.000000,0.1010,-5.634,0.0658,0.412,106.966,2019,"[dance pop, pop, post-teen pop]",dance/electronic,Ariana Grande
5p7ujcrUXASCNwRaWNHR1C,Found you when your heart was broke. I filled ...,1,Without Me,87,1,0.29700,0.752,0.488,0.000009,0.0936,-7.050,0.0705,0.533,136.041,2018,"[dance pop, electropop, etherpop, indie poptim...",dance/electronic,Halsey
2xLMifQCjDGFmkHkpNLD9h,"Astro, yeah. Sun is down, freezin' cold. That'...",1,SICKO MODE,85,1,0.00513,0.834,0.730,0.000000,0.1240,-3.714,0.2220,0.446,155.008,2018,"[pop, pop rap, rap]",pop rap,Travis Scott
1rqqCSm0Qe4I9rUvWncaom,"High, high hopes. Had to have high, high hopes...",0,High Hopes,86,1,0.19300,0.579,0.904,0.000000,0.0640,-2.729,0.0618,0.681,82.014,2018,"[baroque pop, emo, modern rock, pop punk]",pop,Panic! At The Disco
0bYg9bo50gSsH3LtXe2SQn,I-I-I don't want a lot for Christmas. There is...,0,All I Want for Christmas Is You,63,1,0.16400,0.335,0.625,0.000000,0.0708,-7.462,0.0386,0.346,150.277,1994,18,christmas,Mariah Carey


# clustering
https://github.com/UKPLab/sentence-transformers/blob/master/examples/applications/clustering/fast_clustering.py

In [2]:
corpus_sentences = list(df_clean["lyrics"])

In [6]:
from sentence_transformers import SentenceTransformer, util


# load model
mpnet_base = SentenceTransformer('all-mpnet-base-v2')    # max length 384

In [45]:
import time

num_sentences = 5000

subset = corpus_sentences[:num_sentences]

start_time = time.time()
corpus_embeddings = mpnet_base.encode(subset, batch_size=64, show_progress_bar=True, convert_to_tensor=True)
proc_time = time.time() - start_time
print(f"Time for computing embeddings : {proc_time} seconds")
print(f"{proc_time / num_sentences} seconds per song")

Batches:   0%|          | 0/79 [00:00<?, ?it/s]

Time for computing embeddings : 116.98540782928467 seconds
0.023397081565856934 seconds per song


In [15]:
corpus_embeddings.shape

torch.Size([1000, 768])

In [42]:
print("Start clustering")
start_time = time.time()

min_community_size = 5
threshold = 0.75

#Two parameters to tune:
#min_cluster_size: Only consider cluster that have at least X elements
#threshold: Consider sentence pairs with a cosine-similarity larger than threshold as similar
clusters = util.community_detection(corpus_embeddings, min_community_size=min_community_size, threshold=threshold)

print(f"number of clusters : {len(clusters)}")
n_clustered_songs = 0
for c in clusters:
    n_clustered_songs += len(c)
print(f"Number of unclustered songs : {num_sentences - n_clustered_songs}")
print("Clustering done after {:.2f} sec".format(time.time() - start_time))

Start clustering
number of clusters : 13
Number of unclustered songs : 740
Clustering done after 0.11 sec


In [44]:
#Print for all clusters the top 3 and bottom 3 elements
max_len = 300
for i, cluster in enumerate(clusters):
    print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
    for sentence_id in cluster[0:3]:
        print("\t", corpus_sentences[sentence_id][:max_len])
#     print("\t", "...")
#     for sentence_id in cluster[-3:]:
#         print("\t", corpus_sentences[sentence_id][:max_len])


Cluster 1, #126 Elements 
	 Look, my bitches all bad, my niggas all real. I ride on his dick in some big tall heels. Big fat checks, big large bills. Front, I'll flip like ten cartwheels. Cold ass bitch, I give broads chills. Ten different looks and my looks all kill. I kiss him in the mouth, I feel all grills. He eat in the c
	 Hey, oh, there it is, there it is, there it is. There it is, there it is (Rich Homie, baby!). . Walked in, thirty thousand in my pocket. Had them niggas just like, Ooh, ooh, ooh. 50,000 dollars on your bitch, wanna fuck me. Got her screamin' like, Ooh, ooh, ooh, ooh. Cause I got a check, nigga wanna
	 Thugger Thugger baby. Fuck all that let's get to it. What lil' shawty say?. She suck on that dick on the plane and I just called her airhead (Woo! Hee!). I just went hunting, I found me a rabbit, I picked out the carrots (Picked out the carrots). I'm just tired of smoking kushy, I need some Moonrock

Cluster 2, #64 Elements 
	 I realized, I belong to you. I feel 